In [25]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import os
import random

In [27]:
def test(i):
    os.system(f'sleep {i};')
    
    return 1 / i

In [28]:
def test1(i):
    """this function ...
    i > 10, i < 20"""
    os.system(f'sleep {i};')
    
    return 1 / (i+1)

In [32]:
help(test1)

Help on function test1 in module __main__:

test1(i)
    this function ...
    i > 10, i < 20



In [29]:
pool = ThreadPoolExecutor(20, thread_name_prefix='test')

In [35]:
[e for e in dir(ThreadPoolExecutor) if not e.startswith('_')]

['map', 'shutdown', 'submit']

In [17]:
vars(pool)

{'_max_workers': 20,
 '_work_queue': <_queue.SimpleQueue at 0x7fcd89efc400>,
 '_idle_semaphore': <threading.Semaphore at 0x7fcd8a563850>,
 '_threads': set(),
 '_broken': False,
 '_shutdown': False,
 '_shutdown_lock': <unlocked _thread.lock object at 0x7fcd801c5bc0>,
 '_thread_name_prefix': 'test',
 '_initializer': None,
 '_initargs': ()}

In [36]:
help(pool.submit)

Help on method submit in module concurrent.futures.thread:

submit(fn, /, *args, **kwargs) method of concurrent.futures.thread.ThreadPoolExecutor instance
    Submits a callable to be executed with the given arguments.
    
    Schedules the callable to be executed as fn(*args, **kwargs) and returns
    a Future instance representing the execution of the callable.
    
    Returns:
        A Future representing the given call.



In [37]:
%%time

futures = []

for i in range(20):
    f = pool.submit(test, i=i)
    futures.append(f)

CPU times: user 13.3 ms, sys: 8.03 ms, total: 21.3 ms
Wall time: 35.7 ms


In [40]:
1+1

2

In [47]:
dir(futures[-2])

['_Future__get_result',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_condition',
 '_done_callbacks',
 '_exception',
 '_invoke_callbacks',
 '_result',
 '_state',
 '_waiters',
 'add_done_callback',
 'cancel',
 'cancelled',
 'done',
 'exception',
 'result',
 'running',
 'set_exception',
 'set_result',
 'set_running_or_notify_cancel']

In [48]:
[f for f in dir(futures[-1]) if not f.startswith('_')]

['add_done_callback',
 'cancel',
 'cancelled',
 'done',
 'exception',
 'result',
 'running',
 'set_exception',
 'set_result',
 'set_running_or_notify_cancel']

In [49]:
help(f.add_done_callback)

Help on method add_done_callback in module concurrent.futures._base:

add_done_callback(fn) method of concurrent.futures._base.Future instance
    Attaches a callable that will be called when the future finishes.
    
    Args:
        fn: A callable that will be called with this future as its only
            argument when the future completes or is cancelled. The callable
            will always be called by a thread in the same process in which
            it was added. If the future has already completed or been
            cancelled then the callable will be called immediately. These
            callables are called in the order that they were added.



In [58]:
f = futures[-1]

In [59]:
f.done()

True

In [60]:
f.exception()

In [61]:
f.result()

0.05263157894736842

In [70]:
%%time

futures = []

for i in range(20):
    # print(i)
    f = pool.submit(test, i=i)
    futures.append(f)

CPU times: user 431 µs, sys: 15 µs, total: 446 µs
Wall time: 463 µs


In [68]:
%%time

futures = []

for i in range(20):
    # print(i)
    f = pool.submit(test, i=i)
    futures.append(f)

res = []

for f in futures:
    
    try:
        res.append(f.result())
    except:
        res.append(None)

CPU times: user 9.93 ms, sys: 3.84 ms, total: 13.8 ms
Wall time: 19 s


In [67]:
futures[-1].result()

0.05263157894736842

In [77]:
1+1

2

In [78]:
res

[None,
 1.0,
 0.5,
 0.3333333333333333,
 0.25,
 0.2,
 0.16666666666666666,
 0.14285714285714285,
 0.125,
 0.1111111111111111,
 0.1,
 0.09090909090909091,
 0.08333333333333333,
 0.07692307692307693,
 0.07142857142857142,
 0.06666666666666667,
 0.0625,
 0.058823529411764705,
 0.05555555555555555,
 0.05263157894736842]

In [79]:
futures[0].exception()

ZeroDivisionError('division by zero')

In [ ]:
vars(pool)

In [48]:
help(pool.map)

Help on method map in module concurrent.futures._base:

map(fn, *iterables, timeout=None, chunksize=1) method of concurrent.futures.thread.ThreadPoolExecutor instance
    Returns an iterator equivalent to map(fn, iter).
    
    Args:
        fn: A callable that will take as many arguments as there are
            passed iterables.
        timeout: The maximum number of seconds to wait. If None, then there
            is no limit on the wait time.
        chunksize: The size of the chunks the iterable will be broken into
            before being passed to a child process. This argument is only
            used by ProcessPoolExecutor; it is ignored by
            ThreadPoolExecutor.
    
    Returns:
        An iterator equivalent to: map(func, *iterables) but the calls may
        be evaluated out-of-order.
    
    Raises:
        TimeoutError: If the entire result iterator could not be generated
            before the given timeout.
        Exception: If fn(*args) raises for any valu

In [92]:
%%time 
res = pool.map(test1, [2] * 200, timeout=None, chunksize=1)

CPU times: user 2.8 ms, sys: 0 ns, total: 2.8 ms
Wall time: 2.81 ms


In [93]:
pool._work_queue

In [94]:
res

<generator object Executor.map.<locals>.result_iterator at 0x7f622841e180>

In [95]:
%%time
list(res)

CPU times: user 22.2 ms, sys: 468 µs, total: 22.7 ms
Wall time: 5.81 s


[0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333

In [66]:
def test2(i):
    os.system('sleep 2')
    res[i] = 1 / i

In [67]:
res = {}

for i in range(100):
    pool.submit(test2, i=random.randint(1, 200))

In [133]:
len(res), res

(76,
 {144: 0.006944444444444444,
  192: 0.005208333333333333,
  39: 0.02564102564102564,
  110: 0.00909090909090909,
  32: 0.03125,
  59: 0.01694915254237288,
  123: 0.008130081300813009,
  81: 0.012345679012345678,
  94: 0.010638297872340425,
  116: 0.008620689655172414,
  153: 0.006535947712418301,
  104: 0.009615384615384616,
  73: 0.0136986301369863,
  15: 0.06666666666666667,
  166: 0.006024096385542169,
  61: 0.01639344262295082,
  106: 0.009433962264150943,
  50: 0.02,
  111: 0.009009009009009009,
  25: 0.04,
  28: 0.03571428571428571,
  10: 0.1,
  138: 0.007246376811594203,
  57: 0.017543859649122806,
  198: 0.005050505050505051,
  64: 0.015625,
  167: 0.005988023952095809,
  150: 0.006666666666666667,
  131: 0.007633587786259542,
  177: 0.005649717514124294,
  72: 0.013888888888888888,
  85: 0.011764705882352941,
  195: 0.005128205128205128,
  12: 0.08333333333333333,
  160: 0.00625,
  83: 0.012048192771084338,
  65: 0.015384615384615385,
  109: 0.009174311926605505,
  69: 0.

In [112]:
vars(pool)

{'_max_workers': 20,
 '_work_queue': <_queue.SimpleQueue at 0x7fcd89efc400>,
 '_idle_semaphore': <threading.Semaphore at 0x7fcd8a563850>,
 '_threads': {<Thread(test_0, started 140520181458496)>,
  <Thread(test_1, started 140520173065792)>,
  <Thread(test_10, started 140519552300608)>,
  <Thread(test_11, started 140519543907904)>,
  <Thread(test_12, started 140519535515200)>,
  <Thread(test_13, started 140519049000512)>,
  <Thread(test_14, started 140519040607808)>,
  <Thread(test_15, started 140519032215104)>,
  <Thread(test_16, started 140519023822400)>,
  <Thread(test_17, started 140519015429696)>,
  <Thread(test_18, started 140519007036992)>,
  <Thread(test_19, started 140518998644288)>,
  <Thread(test_2, started 140520164673088)>,
  <Thread(test_3, started 140520156280384)>,
  <Thread(test_4, started 140520146839104)>,
  <Thread(test_5, started 140520138446400)>,
  <Thread(test_6, started 140519585871424)>,
  <Thread(test_7, started 140519577478720)>,
  <Thread(test_8, started 1405

In [96]:
import pymysql
import uuid
import json

In [97]:
help(pymysql.connect)

Help on class Connection in module pymysql.connections:

class Connection(builtins.object)
 |  Connection(*, user=None, password='', host=None, database=None, unix_socket=None, port=0, charset='', sql_mode=None, read_default_file=None, conv=None, use_unicode=True, client_flag=0, cursorclass=<class 'pymysql.cursors.Cursor'>, init_command=None, connect_timeout=10, read_default_group=None, autocommit=False, local_infile=False, max_allowed_packet=16777216, defer_connect=False, auth_plugin_map=None, read_timeout=None, write_timeout=None, bind_address=None, binary_prefix=False, program_name=None, server_public_key=None, ssl=None, ssl_ca=None, ssl_cert=None, ssl_disabled=None, ssl_key=None, ssl_verify_cert=None, ssl_verify_identity=None, compress=None, named_pipe=None, passwd=None, db=None)
 |  
 |  Representation of a socket with a mysql server.
 |  
 |  The proper way to get an instance of this class is to call
 |  connect().
 |  
 |  Establish a connection to the MySQL database. Accepts se

In [98]:
conn = {
    'host': 'localhost',
    'port': 3306,
    'user': 'alex',
    'password': '17012011',
    'database': 'test',
}

In [99]:
def insert_row_to_db(key, data):
    with pymysql.connect(**conn) as con:
        with con.cursor() as cur:
            cur.execute("insert into test.test (uid, data) values (%s, %s)", (key, json.dumps(data)))
            id_ = cur.lastrowid
        con.commit()
    return id_, key

In [7]:
str(uuid.uuid4())

'cd73c9e1-788e-415c-b45a-73828821f318'

In [100]:
def mock_data():
    return {
        'value': random.randint(1,200)
    }

In [107]:
mock_data()

{'value': 154}

In [108]:
key = str(uuid.uuid4())
data = mock_data()

In [109]:
%%time
insert_row_to_db(key, data)

CPU times: user 2.51 ms, sys: 3.88 ms, total: 6.38 ms
Wall time: 9.38 ms


(1, 'b6ed0c48-8b9b-44d9-a0a4-9edfe33eaa36')

In [112]:
db_conn_pool = ThreadPoolExecutor(50, thread_name_prefix='db_conn')

In [114]:
%%time

futures = []

for i in range(10000):
    key = str(uuid.uuid4())
    data = mock_data()
    f = db_conn_pool.submit(insert_row_to_db, key=key, data=data)
    futures.append(f)
    # insert_row_to_db(key, data)

futures[-1].result()

CPU times: user 13.5 s, sys: 12.9 s, total: 26.3 s
Wall time: 17.4 s


(30001, '6214ade8-ebcf-4bd2-8d14-ff98216a9dbb')

In [136]:
1+1

2

In [115]:
futures[-1].done()

True

In [117]:
futures[-2].result()

(29998, '71e794f3-38d5-4171-8ed3-d69ad3e94348')

In [118]:
futures[-1].result()

(30001, '6214ade8-ebcf-4bd2-8d14-ff98216a9dbb')

In [142]:
35008 - 34996

12

In [116]:
key

'5e3b4b57-e6b6-4d9e-86c5-622f3fd59c96'

In [117]:
i

9999

In [118]:
f = db_conn_pool.submit(insert_row_to_db, key=key, data=data)

In [119]:
# другой код

inserted_data = f.result()
inserted_data

(25007, '5e3b4b57-e6b6-4d9e-86c5-622f3fd59c96')

In [163]:
help(db_conn_pool.shutdown)

Help on method shutdown in module concurrent.futures.thread:

shutdown(wait=True, *, cancel_futures=False) method of concurrent.futures.thread.ThreadPoolExecutor instance
    Clean-up the resources associated with the Executor.
    
    It is safe to call this method several times. Otherwise, no other
    methods can be called after this one.
    
    Args:
        wait: If True then shutdown will not return until all running
            futures have finished executing and the resources used by the
            executor have been reclaimed.
        cancel_futures: If True then shutdown will cancel all pending
            futures. Futures that are completed or running will not be
            cancelled.



In [119]:
db_conn_pool.shutdown()

In [120]:
p_pool = ProcessPoolExecutor(8)

In [121]:
[f for f in dir(p_pool) if not f.startswith('_')]

['map', 'shutdown', 'submit']

In [122]:
help(p_pool.submit)

Help on method submit in module concurrent.futures.process:

submit(fn, /, *args, **kwargs) method of concurrent.futures.process.ProcessPoolExecutor instance
    Submits a callable to be executed with the given arguments.
    
    Schedules the callable to be executed as fn(*args, **kwargs) and returns
    a Future instance representing the execution of the callable.
    
    Returns:
        A Future representing the given call.



In [125]:
help(p_pool.map)

Help on method map in module concurrent.futures.process:

map(fn, *iterables, timeout=None, chunksize=1) method of concurrent.futures.process.ProcessPoolExecutor instance
    Returns an iterator equivalent to map(fn, iter).
    
    Args:
        fn: A callable that will take as many arguments as there are
            passed iterables.
        timeout: The maximum number of seconds to wait. If None, then there
            is no limit on the wait time.
        chunksize: If greater than one, the iterables will be chopped into
            chunks of size chunksize and submitted to the process pool.
            If set to one, the items in the list will be sent one at a time.
    
    Returns:
        An iterator equivalent to: map(func, *iterables) but the calls may
        be evaluated out-of-order.
    
    Raises:
        TimeoutError: If the entire result iterator could not be generated
            before the given timeout.
        Exception: If fn(*args) raises for any values.



In [123]:
def fibb(x):
    if x in [1, 2]:
        return 1
    else:
        return fibb(x-1) + fibb(x-2)

In [170]:
%%time
fibb(39)

CPU times: user 18.7 s, sys: 0 ns, total: 18.7 s
Wall time: 18.9 s


63245986

In [124]:
t_pool = ThreadPoolExecutor(8)

In [172]:
%%time

for i in range(5):
    f = t_pool.submit(fibb, x=39)
    
f.result()

CPU times: user 1min 47s, sys: 1.4 s, total: 1min 48s
Wall time: 1min 48s


63245986

In [173]:
%%time

for i in range(5):
    f = p_pool.submit(fibb, x=39)
    
f.result()

CPU times: user 21.3 ms, sys: 88.8 ms, total: 110 ms
Wall time: 27.2 s


63245986

In [9]:
from functools import lru_cache

In [16]:
@lru_cache(100000)
def fibb_m(x):
    if x in [1, 2]:
        return 1
    else:
        return fibb_m(x-1) + fibb_m(x-2)

In [20]:
%%time

fibb_m(400)

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 10.5 µs


176023680645013966468226945392411250770384383304492191886725992896575345044216019675

In [182]:
%%time

fibb_m(38)

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 14.3 µs


39088169

In [147]:
%%time

fibb_m(39)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 10.5 µs


63245986

In [148]:
%%time

fibb_m(38)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 9.06 µs


39088169

In [ ]:
f(2)

In [22]:
class memoize:
    def __init__(self, func):
        self.cache = {}
        self.func = func
        
    def __call__(self, x):
        # можно делать свои операции - обращаться за кэшем в БД, memcached, собирать запчасти из разных углов
        try:
            return self.cache[x]
        except KeyError:
            self.cache[x] = self.func(x)
            return self.cache[x]

In [23]:
@memoize
def fibb_2(x):
    if x in [1, 2]:
        return 1
    else:
        return fibb_2(x-1) + fibb_2(x-2)

In [24]:
%%time
fibb_2(32)

CPU times: user 132 µs, sys: 3 µs, total: 135 µs
Wall time: 140 µs


2178309

In [ ]:
help(ProcessPoolExecutor)